In [1]:
import hashlib
import inspect

In [2]:
def fdigest(filename):
    f = open(filename,mode='rb')
    m = hashlib.sha256(f.read())
    f.close()
    return m.hexdigest()

In [3]:
h = fdigest('Loads.ipynb')
h

'd5b691758babfd8d3ddd6ad3842c08c87c825361fd9eaf8ec904ac0a7faca53d'

In [4]:
def extend(new):
    name = new.__name__
    old = eval(name)
    print name,id(old),old
    print id(new), new
    for a,v in inspect.getmembers(new):
        if not a.startswith('_'):
            setattr(old,a,v)
            print 'Set', a
    return old

In [5]:
class Foo(object):
    def one(self):
        return 1

In [6]:
id(Foo), Foo

(3042152964L, __main__.Foo)

In [7]:
@extend
class Foo(object):
    def two(self):
        return 2

Foo 3042152964 <class '__main__.Foo'>
3042153436 <class '__main__.Foo'>
Set two


In [8]:
@extend
class Foo:
    def three(self):
        return 3

Foo 3042152964 <class '__main__.Foo'>
3042832076 __main__.Foo
Set three


In [9]:
f = Foo()

In [10]:
f.one()

1

In [11]:
f.two()

TypeError: unbound method two() must be called with Foo instance as first argument (got nothing instead)

In [12]:
f.one

<bound method Foo.one of <__main__.Foo object at 0xb5532f2c>>

In [13]:
f.two

<unbound method Foo.two>

In [14]:
Foo.one

<unbound method Foo.one>

In [15]:
Foo.two

<unbound method Foo.two>

In [16]:
def extend2(old):
    """A decorator to extend class definitions.  Caution, only
    works with normal methods; does not work with properties,
    class mewthods or static methods in the new class fragment."""
    def _fn(new,old=old):
        for a,v in inspect.getmembers(new):
            if not a.startswith('_'):
                setattr(old,a,v)
        return old
    return _fn

In [17]:
class Bar(object):
    def one(s):
        return s,1

In [18]:
@extend2(Bar)
class Bar:
    def two(s):
        return s,2

In [19]:
b = Bar()

In [20]:
b.one()

(<__main__.Bar at 0xb4b6662c>, 1)

In [21]:
b.two()

TypeError: unbound method two() must be called with Bar instance as first argument (got nothing instead)

In [24]:
Bar.one

<unbound method Bar.one>

In [26]:
Bar.two

<unbound method Bar.two>

In [30]:
Bar.one.im_class

__main__.Bar

In [31]:
Bar.two.im_class

<class __main__.Bar at 0xb4b6a08c>

In [32]:
Bar.two.im_class = Bar.one.im_class

TypeError: readonly attribute

In [33]:
import types

In [34]:
dir(types)

['BooleanType',
 'BufferType',
 'BuiltinFunctionType',
 'BuiltinMethodType',
 'ClassType',
 'CodeType',
 'ComplexType',
 'DictProxyType',
 'DictType',
 'DictionaryType',
 'EllipsisType',
 'FileType',
 'FloatType',
 'FrameType',
 'FunctionType',
 'GeneratorType',
 'GetSetDescriptorType',
 'InstanceType',
 'IntType',
 'LambdaType',
 'ListType',
 'LongType',
 'MemberDescriptorType',
 'MethodType',
 'ModuleType',
 'NoneType',
 'NotImplementedType',
 'ObjectType',
 'SliceType',
 'StringType',
 'StringTypes',
 'TracebackType',
 'TupleType',
 'TypeType',
 'UnboundMethodType',
 'UnicodeType',
 'XRangeType',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__']

In [35]:
class Bar2(object):
    def one(s):
        return s,1

In [36]:
@extend2(Bar2)
class Bar3:
    def two(s):
        return s,2

In [37]:
Bar2

__main__.Bar2

In [38]:
Bar3

__main__.Bar2

In [39]:
Bar2 is Bar3

True

In [40]:
b2 = Bar2()

In [41]:
b2.one()

(<__main__.Bar2 at 0xb4b7e18c>, 1)

In [42]:
b2.two()

TypeError: unbound method two() must be called with Bar3 instance as first argument (got nothing instead)

In [43]:
Bar2.one

<unbound method Bar2.one>

In [44]:
Bar2.two

<unbound method Bar3.two>